In [84]:
# Import modules and packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

%matplotlib inline

In [85]:
dataset_train = pd.read_csv('/Users/dennis/kaggle/input/predictive_maintenance/pump_data.csv', sep=';', low_memory=False , decimal=',')

In [86]:
dataset_train.head()

,Unnamed: 0,Date,03:I3SROE,03:T3362,03:PI3061A,03:PI3061B,03:PI3061C,03:PD3066,03:T3492,03:T3358,...,03:E3851,03:T3632,03:T3633,03:T3634,03:FC3195,03:FV3195,03:FC3197,03:FV3197,03:FC3199,03:FV3199
0,0,01.01.19 00:00,"890,187","77,923","3,025","3,043","3,055",No Data,"40,456","97,602",...,"0,156","16,62","16,661","13,997","358,322",100,"362,52",100,"392,51",100
1,1,01.01.19 00:10,"890,262","77,893","3,024","3,043","3,053",No Data,"40,415","97,178",...,"0,157","16,675","16,676","13,985","358,897",100,"362,505",100,"393,009",100
2,2,01.01.19 00:20,"890,337","77,863","3,024","3,043","3,051",No Data,"40,375","97,215",...,"0,158","16,711","16,726","14,02","358,984",100,"362,305",100,"392,767",100
3,3,01.01.19 00:30,"890,294","77,833","3,023","3,043","3,051",No Data,"40,41","97,602",...,"0,159","16,716","16,784","14,035","362,205",100,"361,171",100,"391,512",100
4,4,01.01.19 00:40,"890,239","77,823","3,023","3,043","3,052",No Data,"40,41","97,438",...,"0,16","16,75","16,815","14,065","361,954",100,"360,852",100,"390,736",100


In [87]:
dataset_train = dataset_train.replace(to_replace='N_LAUF', value='0')
dataset_train = dataset_train.replace(to_replace='LAUF', value='1')
dataset_train = dataset_train.replace(to_replace='No Data', value='-1001')
dataset_train = dataset_train.replace(to_replace='Over Range', value='-1002')
dataset_train = dataset_train.replace(to_replace='I/O Timeout', value='-1003')
dataset_train = dataset_train.replace(to_replace='Calc Failed', value='-1004')
dataset_train = dataset_train.replace(to_replace='Error', value='-1005')
dataset_train = dataset_train.replace(to_replace='Configure', value='-1006')
dataset_train = dataset_train.replace(to_replace='Scan Off', value='-1007')
dataset_train = dataset_train.replace(to_replace='Comm Fail', value='-1008')
dataset_train = dataset_train.replace(to_replace='Shutdown', value='-1009')
dataset_train = dataset_train.replace(to_replace='NaN', value='-1010')
dataset_train = dataset_train.replace(to_replace='Bad Input', value='-1011')
dataset_train = dataset_train.replace(to_replace='Bad', value='-1012')

In [88]:
dataset_train = dataset_train.drop(['Unnamed: 0'], axis = 1)
dataset_train = dataset_train.drop(['03:FV3195'], axis = 1)
dataset_train = dataset_train.drop(['03:FV3197'], axis = 1)
dataset_train = dataset_train.drop(['03:FV3199'], axis = 1)

In [89]:
dataset_train = dataset_train[1:].replace(',','.', regex=True)

In [90]:
for column in dataset_train[1:]:
    dataset_train[column] = pd.to_numeric(dataset_train[column], errors='coerce')

In [91]:
dataset_train['Durchsatz'] = dataset_train['03:FC3195'] + dataset_train['03:FC3197'] + dataset_train['03:FC3199']
dataset_train['Durchsatz']

0         1113.352
1         1114.411
2         1114.056
3         1114.888
4         1113.542
            ...   
574000     932.979
574001     933.390
574002     933.553
574003     932.625
574004     944.560
Name: Durchsatz, Length: 574005, dtype: float64

In [92]:
# Select features (columns) to be involved intro training and predictions
cols = list(dataset_train)[1:]

cols

['03:I3SROE',
 '03:T3362',
 '03:PI3061A',
 '03:PI3061B',
 '03:PI3061C',
 '03:PD3066',
 '03:T3492',
 '03:T3358',
 '03:TC3550',
 '03:TC3550A',
 '03:FV3762',
 '03:FC3762',
 '03:TC3730',
 '03:TV3730A',
 '03:TV3730B',
 '03:TV3730C',
 '03:T3731',
 '03:P3649',
 '03:P3065',
 '03:LC3661',
 '03:LI3661A',
 '03:LI3661B',
 '03:LI3661C',
 '03:YV4978',
 '03:FC3075.OP',
 '03:FC3075',
 '03:T3361',
 '03:T3359',
 '03:T3360',
 '03:N3722_1',
 '03:N3721_1',
 '03:N3725_1',
 '03:E3849',
 '03:T3626',
 '03:T3627',
 '03:T3628',
 '03:N3726_1',
 '03:E3850',
 '03:T3629',
 '03:T3630',
 '03:T3631',
 '03:N3723_1',
 '03:N3727_1',
 '03:E3851',
 '03:T3632',
 '03:T3633',
 '03:T3634',
 '03:FC3195',
 '03:FC3197',
 '03:FC3199',
 'Durchsatz']

In [93]:
dataset_train.dtypes

Date            float64
03:I3SROE       float64
03:T3362        float64
03:PI3061A      float64
03:PI3061B      float64
03:PI3061C      float64
03:PD3066       float64
03:T3492        float64
03:T3358        float64
03:TC3550       float64
03:TC3550A      float64
03:FV3762       float64
03:FC3762       float64
03:TC3730       float64
03:TV3730A      float64
03:TV3730B      float64
03:TV3730C      float64
03:T3731        float64
03:P3649        float64
03:P3065        float64
03:LC3661       float64
03:LI3661A      float64
03:LI3661B      float64
03:LI3661C      float64
03:YV4978       float64
03:FC3075.OP    float64
03:FC3075       float64
03:T3361        float64
03:T3359        float64
03:T3360        float64
03:N3722_1      float64
03:N3721_1      float64
03:N3725_1      float64
03:E3849        float64
03:T3626        float64
03:T3627        float64
03:T3628        float64
03:N3726_1      float64
03:E3850        float64
03:T3629        float64
03:T3630        float64
03:T3631        

In [94]:
# Extract dates (will be used in visualization)
datelist_train = list(dataset_train['Date'])
print('Training set shape == {}'.format(dataset_train.shape))
print('All timestamps == {}'.format(len(datelist_train)))
print('Featured selected: {}'.format(cols))

Training set shape == (574005, 52)
All timestamps == 574005
Featured selected: ['03:I3SROE', '03:T3362', '03:PI3061A', '03:PI3061B', '03:PI3061C', '03:PD3066', '03:T3492', '03:T3358', '03:TC3550', '03:TC3550A', '03:FV3762', '03:FC3762', '03:TC3730', '03:TV3730A', '03:TV3730B', '03:TV3730C', '03:T3731', '03:P3649', '03:P3065', '03:LC3661', '03:LI3661A', '03:LI3661B', '03:LI3661C', '03:YV4978', '03:FC3075.OP', '03:FC3075', '03:T3361', '03:T3359', '03:T3360', '03:N3722_1', '03:N3721_1', '03:N3725_1', '03:E3849', '03:T3626', '03:T3627', '03:T3628', '03:N3726_1', '03:E3850', '03:T3629', '03:T3630', '03:T3631', '03:N3723_1', '03:N3727_1', '03:E3851', '03:T3632', '03:T3633', '03:T3634', '03:FC3195', '03:FC3197', '03:FC3199', 'Durchsatz']


In [98]:
# Using multiple features (predictors)
#training_set = dataset_train.as_matrix(columns = cols)
training_set = np.array(dataset_train[cols])

In [99]:
print('Shape of training set == {}.'.format(training_set.shape))
training_set

Shape of training set == (574005, 51).


array([[ 890.187,   77.923,    3.025, ...,  362.52 ,  392.51 , 1113.352],
       [ 890.262,   77.893,    3.024, ...,  362.505,  393.009, 1114.411],
       [ 890.337,   77.863,    3.024, ...,  362.305,  392.767, 1114.056],
       ...,
       [ 750.08 ,   73.618,    2.696, ...,  300.264,  333.121,  933.553],
       [ 751.267,   73.667,    2.699, ...,  299.913,  332.373,  932.625],
       [ 755.64 ,   73.563,    2.701, ...,  305.107,  338.502,  944.56 ]])

In [104]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
training_set_scaled = sc.fit_transform(training_set)

sc_predict = StandardScaler()
sc_predict.fit_transform(training_set[:, 0:1])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/extmath.py:765: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/extmath.py:706: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


array([[ 0.14994906],
       [ 0.15041378],
       [ 0.15087849],
       ...,
       [-0.71817884],
       [-0.71082398],
       [-0.68372809]])

In [107]:
training_set_scaled.shape

(574005, 51)

In [114]:
cols

['03:I3SROE',
 '03:T3362',
 '03:PI3061A',
 '03:PI3061B',
 '03:PI3061C',
 '03:PD3066',
 '03:T3492',
 '03:T3358',
 '03:TC3550',
 '03:TC3550A',
 '03:FV3762',
 '03:FC3762',
 '03:TC3730',
 '03:TV3730A',
 '03:TV3730B',
 '03:TV3730C',
 '03:T3731',
 '03:P3649',
 '03:P3065',
 '03:LC3661',
 '03:LI3661A',
 '03:LI3661B',
 '03:LI3661C',
 '03:YV4978',
 '03:FC3075.OP',
 '03:FC3075',
 '03:T3361',
 '03:T3359',
 '03:T3360',
 '03:N3722_1',
 '03:N3721_1',
 '03:N3725_1',
 '03:E3849',
 '03:T3626',
 '03:T3627',
 '03:T3628',
 '03:N3726_1',
 '03:E3850',
 '03:T3629',
 '03:T3630',
 '03:T3631',
 '03:N3723_1',
 '03:N3727_1',
 '03:E3851',
 '03:T3632',
 '03:T3633',
 '03:T3634',
 '03:FC3195',
 '03:FC3197',
 '03:FC3199',
 'Durchsatz']

In [136]:
train_size = int(len(training_set_scaled) * 0.67)
test_size = len(training_set_scaled) - train_size
train, test = training_set_scaled[0:train_size,:], training_set_scaled[train_size:len(training_set_scaled),:]
print(len(train), len(test))

train.shape

384583 189422


(384583, 51)

In [129]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [135]:
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
trainX.shape

(384581, 1)

In [132]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [138]:
trainX.shape
testX.shape

(189420, 1)

In [146]:
from numpy import hstack
dataset = hstack('03:I3SROE','03:T3362','03:PI3061A')

TypeError: _vhstack_dispatcher() takes 1 positional argument but 3 were given

In [145]:
print(dataset)

['03:I3SROE' '03:T3362' '03:PI3061A' '03:PI3061B' '03:PI3061C' '03:PD3066'
 '03:T3492' '03:T3358' '03:TC3550' '03:TC3550A' '03:FV3762' '03:FC3762'
 '03:TC3730' '03:TV3730A' '03:TV3730B' '03:TV3730C' '03:T3731' '03:P3649'
 '03:P3065' '03:LC3661' '03:LI3661A' '03:LI3661B' '03:LI3661C' '03:YV4978'
 '03:FC3075.OP' '03:FC3075' '03:T3361' '03:T3359' '03:T3360' '03:N3722_1'
 '03:N3721_1' '03:N3725_1' '03:E3849' '03:T3626' '03:T3627' '03:T3628'
 '03:N3726_1' '03:E3850' '03:T3629' '03:T3630' '03:T3631' '03:N3723_1'
 '03:N3727_1' '03:E3851' '03:T3632' '03:T3633' '03:T3634' '03:FC3195'
 '03:FC3197' '03:FC3199' 'Durchsatz']


In [142]:

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [144]:
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

IndexError: too many indices for array